# PandaSDMX

- [Documentazione aggiornata (v1.4.1)](https://pandasdmx.readthedocs.io/en/latest/)
- [Esempio breve (con poche spiegazioni)](https://pandasdmx.readthedocs.io/en/master/example.html)
- [Esempio approfondito (ma non troppo aggiornato)](https://pandasdmx.readthedocs.io/en/latest/walkthrough.html)

## Installazione

- L'ultima versione non funziona con Pydantic 1.8.1 ma richiede 1.7 ([dr-leo/pandaSDMX#204](https://github.com/dr-leo/pandaSDMX/issues/204))

In [1]:
!pip install pandasdmx pydantic==1.7

## Esempio

In [2]:
import pandas
import pandasdmx

# Per type annotations
import pandasdmx.message
import pandasdmx.model
import pandasdmx.source
import pandasdmx.source.estat

/mnt/tera/ext4/code/sdmx-sandbox/venv/lib/python3.9/site-packages/pandasdmx/remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


È possibile selezionare tra più fonti di dati, tra i quali Eurostat (`ESTAT`).

> __Request__: client di comunicazione tra `pandasdmx` e un server di dati come Eurostat

Come prima cosa, è necessario creare un'istanza di `pandasdmx.Request`:

In [3]:
eurostat: pandasdmx.Request = pandasdmx.Request("ESTAT")
eurostat

> __Dataflow__: set di metadati relativi a una misura effettuata (ad esempio, `educ_enrl1ad - Students by ISCED level, study intensity and sex`)

> __Message__: risposta HTTPS ricevuta in seguito a una richiesta effettuata ad un server di dati

Poi, scarichiamo _tutti_ i dataflow disponibili usando `.dataflow()` sul client creato in precedenza per effettuare una richiesta al server Eurostat, creando un `pandasdmx.message.Message`:

In [4]:
all_flows_msg: pandasdmx.message.Message = eurostat.dataflow()
all_flows_msg

<pandasdmx.StructureMessage>
  <Header>
    id: 'IDREF382067'
    prepared: '2021-03-15T01:45:49.005000+00:00'
    receiver: <Agency Unknown>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (6573): DS-018995 DS-022469 DS-032655 DS-043227 DS...
  DataStructureDefinition (6573): DSD_DS-018995 DSD_DS-022469 DSD_DS-03...

> __Series__: una specie di `dict` più veloce e avanzato implementato da `pandas`

PandaSDMX ha la funzionalità che cercavamo di cercare dataset per keyword!

Per effettuare la ricerca, usiamo il metodo `.to_pandas()` per convertire il `Message` in oggetti Python e/o `pandas`, poi usiamo i metodi "nativi" per trovare quello che ci serve:

In [5]:
# Converte i risultati in due Series di pandas, una con i dataflow e una con la loro relativa struttura
_dict: dict[str, pandas.Series] = all_flows_msg.to_pandas()
all_flows: pandas.Series = _dict["dataflow"]
all_structs: pandas.Series = _dict["structure"]
all_flows, all_structs

(DS-018995                               EU trade since 1988 by SITC
 DS-022469         EXTRA EU trade since 1999 by mode of transport...
 DS-032655                                EU trade since 1988 by BEC
 DS-043227                             EFTA trade since 1995 by SITC
 DS-066341         Sold production, exports and imports by PRODCO...
                                         ...                        
 yth_incl_120      Young people living in households with very lo...
 yth_part_010      Frequency of getting together with relatives o...
 yth_part_020      Frequency of contacts with relatives or friend...
 yth_part_030      Participation of young people in activities of...
 yth_volunt_010    Participation of young people in informal volu...
 Length: 6573, dtype: object,
 DSD_DS-018995          
 DSD_DS-022469          
 DSD_DS-032655          
 DSD_DS-043227          
 DSD_DS-066341          
                      ..
 DSD_yth_incl_120       
 DSD_yth_part_010       
 DSD_yth_pa

In [6]:
# Cerchiamo nella Series i allflows la cui descrizione contiene "student"
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
student_flows: pandas.Series = all_flows[all_flows.str.contains("student", case=False)]
student_flows

educ_enrl1ad        Students by ISCED level, study intensity and sex
educ_enrl1at       Students by ISCED level, type of institution a...
educ_enrl1tl                    Students by ISCED level, age and sex
educ_enrl5         Tertiary students (ISCED 5-6) by field of educ...
educ_enrl6         Tertiary students (ISCED 5-6)  non-citizens, n...
educ_enrl8         Tertiary students (ISCED 5-6) by country of ci...
educ_enrllng1      Students in ISCED 1-3 by modern foreign langua...
educ_enrllng2      Students in ISCED 1-3 by number of modern fore...
educ_fiaid                                 Financial aid to students
educ_ilev                  Distribution of pupils/ students by level
educ_iste          Pupil/ student - teacher ratio and average cla...
educ_mofo_dst      Foreign students by level of education and cou...
educ_mofo_fld       Foreign students by level and field of education
educ_mofo_gen         Foreign students by level of education and sex
educ_mofo_orig     Foreign student

Per continuare gli esperimenti, prendiamo il primo dataflow tra quelli contenenti `"student"` nel label:

In [7]:
my_flow_label = student_flows.index[0]
my_flow_label

'educ_enrl1ad'

Usiamo il label per chiamare di nuovo `.dataflow()`, specificando però stavolta il dataflow di cui ci interessano i dettagli:

In [8]:
my_flow_msg: pandasdmx.message.Message = eurostat.dataflow(my_flow_label)
my_flow: pandasdmx.model.DataflowDefinition = my_flow_msg.dataflow[my_flow_label]
my_flow

<DataflowDefinition ESTAT:educ_enrl1ad(1.0): Students by ISCED level, study intensity and sex>

> __Structure__: metadati su come sono strutturate le misure di un dataflow (cosa è stato misurato, quali filtri è possibile applicare, note, etc)

_Particolarità di Eurostat: la structure va richiesta separatamente dal dataflow, in quanto tutti i campi a parte `id` di `dataflow.structure` sono sempre vuoti._

Scopriamo prima il label della structure, poi scarichiamo da Eurostat la structure del dataflow che ci interessa con il metodo `.datastructure()`:

In [9]:
my_struct_label: pandasdmx.source.DataStructureDefinition = my_flow.structure.id
my_struct_msg: pandasdmx.message.Message = eurostat.datastructure(my_struct_label)
my_struct: pandasdmx.source.DataStructureDefinition = my_struct_msg.structure[my_struct_label]
my_struct

<DataStructureDefinition ESTAT:DSD_educ_enrl1ad(1.0): DSWS Data Structure Definition>

Ispezioniamo la structure che abbiamo scaricato, visualizzandola contemporaneamente [sul Data Explorer di Eurostat](https://ec.europa.eu/eurostat/databrowser/view/educ_enrl1ad/default/table?lang=en)

> __Measures__: valori aggregati relativi alle misure effettuate, simili a `COUNT(*)` dell'SQL

> __Dimensions__: filtri applicabili ai dati raccolti in modo simile all'`HAVING` dell'SQL

> __Attributes__: ???

> __Annotations__: commenti che possono essere aggiunti al dataflow

In [10]:
my_struct.annotations, my_struct.measures, my_struct.attributes, my_struct.dimensions

([],
 <MeasureDescriptor: <PrimaryMeasure OBS_VALUE>>,
 <AttributeDescriptor: <DataAttribute OBS_FLAG>; <DataAttribute OBS_STATUS>>,
 <DimensionDescriptor: <Dimension FREQ>; <Dimension UNIT>; <Dimension ISCED97>; <Dimension SEX>; <Dimension WORKTIME>; <Dimension GEO>; <TimeDimension TIME_PERIOD>>)

Infine, richiediamo i dati da Eurostat, limitandoli a quelli dell'`IT`alia dal 2010 in poi e selezionando solo il `WORKTIME` `TOTAL`, e convertiamoli in una Series multi-chiave:

In [28]:
my_data_msg: pandasdmx.message.Message = eurostat.data(my_flow_label, key={"GEO": "IT", "WORKTIME": "TOTAL"}, params={"startPeriod": "2010"})
my_data: pandas.Series = my_data_msg.to_pandas()
my_data

FREQ  UNIT  ISCED97  SEX  WORKTIME  GEO  TIME_PERIOD
A     NR    ED0      F    TOTAL     IT   2010           808706.0
                                         2011           811615.0
                                         2012           815656.0
                     M    TOTAL     IT   2010           872281.0
                                         2011           876225.0
                                                          ...   
            UNK      M    TOTAL     IT   2011                NaN
                                         2012                NaN
                     T    TOTAL     IT   2010                NaN
                                         2011                NaN
                                         2012                NaN
Name: value, Length: 279, dtype: float64

Abbiamo ricevuto i dati, e possiamo manipolarli come una qualsiasi series di `pandas` (le quali sono molto simili a tabelle SQL in-memory):

In [30]:
# Il numero di studenti [M]aschi, [F]emmine e [T]otali in Italia nel [2010], [2011] e [2012]
my_data.groupby(["SEX", "TIME_PERIOD"]).first()

SEX  TIME_PERIOD
F    2010            808706.0
     2011            811615.0
     2012            815656.0
M    2010            872281.0
     2011            876225.0
     2012            879256.0
T    2010           1680987.0
     2011           1687840.0
     2012           1694912.0
Name: value, dtype: float64

## Sorgenti dati

Tra le sorgenti di dati di cui abbiamo parlato, sono [completamente supportate](https://pandasdmx.readthedocs.io/en/latest/sources.html):

- `ESTAT` - Eurostat
- `ISTAT` - ISTAT

Queste sorgenti non supportano lo standard `SDMX-MD` ma solo lo standard `SDMX-JSON`, che [non supporta query di metadati e struttura](https://pandasdmx.readthedocs.io/en/latest/sources.html#data-source-limitations):

- `OECD` - Organisation for Economic Cooperation and Development

## Archiviazione dati

Se si vogliono replicare dati provenienti da queste fonti, si potrebbe usare tranquillamente un database **relazionale** (SQL) le cui tabelle sono generate a runtime in base alla struttura del dataflow desiderato.

[SQLAlchemy](https://www.sqlalchemy.org/) potrebbe essere utile in questo caso; non sono particolarmente familiare con l'[ORM di Django](https://docs.djangoproject.com/en/3.1/topics/db/models/), ma sembrano molto simili (anche se [si direbbe che SQLAlchemy supporti query più complesse](https://stackoverflow.com/questions/18199053/example-of-what-sqlalchemy-can-do-and-django-orm-cannot)).